In [10]:
import pandas as pd
import numpy as np
import re
import sys
from sklearn import *
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer 
from sklearn.ensemble import RandomForestClassifier

In [11]:
#Loading dataset
data_path = r"C:/Users/eunic/OneDrive/Desktop/trainingObamaRomneytweets.xlsx"
data_romney = pd.read_excel(data_path, sheetname = 'Romney')

# Removing rows with classes other than 1,-1,0
data_romney = data_romney[(data_romney['Class'].isin((1,-1,0)))]

stop_words = set(stopwords.words('english')) 
ps = PorterStemmer() 
lemmatizer = WordNetLemmatizer() 
lmtzr = WordNetLemmatizer()

#predefined function for cleaning the text
def clean_text(row):
    tweet = str(row)
    cleanr = re.compile('(</?[a-zA-Z]+>|https?:\/\/[^\s]*|(^|\s)RT(\s|$)|@[^\s]+|\d+)')
    cleantext = re.sub(cleanr, ' ', tweet)
    cleantext = re.sub('(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)',' ',cleantext)
    cleantext = re.sub('[^\sa-zA-Z]+','',cleantext)
    cleantext = re.sub('\s+',' ',cleantext)       
    lower_case = cleantext.lower() 
    tweet = str(lower_case)
    tweet = tweet.replace("'", "")
    tweet = tweet.replace("\"","")
    tweet = lmtzr.lemmatize(tweet) 
    return tweet

#Calculating tf-idf for each tweet
def vectorize_data(tweets):
    vectorizer = CountVectorizer()
    vectorizer.fit(tweets)
    vector = vectorizer.transform(tweets)
    return vector

#Extracting only the class and annotation from given file
new_data_romney= data_romney[['Class','Anootated tweet']]

#Creating a column with pre processed tweets
new_data_romney['Cleaned_tweet'] = new_data_romney['Anootated tweet'].apply(clean_text)

# converting the tweets and class to list form
cleaned_romney_tweets = new_data_romney['Cleaned_tweet'].tolist()
romney_class = new_data_romney['Class'].tolist()

#Vectorizing the text data
vector_romney_tweets = vectorize_data(cleaned_romney_tweets)

C:\Users\eunic\Anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
C:\Users\eunic\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
# RandomForest Tree Classifier
print(" RandomForest Classifier")
model = RandomForestClassifier(criterion='entropy' , n_jobs = 10, n_estimators = 100)
folds = KFold(n_splits = 10, shuffle = True)

#Romney
predicts1= cross_val_predict(model, vector_romney_tweets, romney_class, cv = folds)
classes = [1,0,-1]
accScore_romney = metrics.accuracy_score(romney_class,predicts1)
precision = metrics.precision_score(romney_class,predicts1,average=None,labels=classes)
recall = metrics.recall_score(romney_class,predicts1,average=None,labels=classes)
f1Score = metrics.f1_score(romney_class,predicts1,average=None,labels=classes)
print("Romney:  Acurracy: ",accScore_romney)
print("Precision :", precision)
print("Recall", recall)
print("F1-Score", f1Score)

 RandomForest Classifier
Romney:  Acurracy:  0.5940155807365439
Precision : [0.65591398 0.49956179 0.61256014]
Recall [0.34046512 0.33928571 0.83615624]
F1-Score [0.44825475 0.40411202 0.70710319]
